<b><font size=6 color=mediumblue>Laboratory III – Part B</font></b>

### <font color='green'>Learning Goals</font>
By the end of this notebook, you will
1. Make plots of the meridional overturning circulation in the Indo-Pacific and Atlantic
2. Make a plot of the global meridional ocean heat transport
3. Jot down some basic notes about what you see in your figures to prepare you for your lab report

# I) Gathering what you need

## 1. Getting Started
You'll be working with the same python packages and the same data you did in Part A. You will need to import those packages and read in the data on your own below. Make sure all your plots for this part have titles, x and y axis labels, and colorbars or legends. Each plot should also have appropriate units. 

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean.cm as cmo
import cartopy.crs as ccrs

ds = xr.open_dataset('SODA3.15.2_2024.nc')
mask = xr.open_dataset('basin_mask.nc')
ds['basin_mask'] = mask['basin_mask']
ds

---

# II): Making plots of the data

## 3. Overturning Circulation
In this section you'll make two plots:
1. The streamfunction (in Sv) of the time mean meridional overturning circulation in the Atlantic (only from 35˚S to 70˚N)
2. The streamfunction (in Sv) of the time mean meridional overturning circulation in the Indo-Pacific (only from 30˚S to 70˚N)

Recall that the streamfunction for the MOC is <br>

$$\Psi(y, z) = \int_{0}^{z} \left( \int_{xe}^{xw} v(x, y, z')\, dx \right) dz'$$

For this part, you'll need to use the ```basin_mask``` variable in the dataset. The basin_mask variable is a 2D horizontal field that has preassigned integer values for each ocean basin. For the Atlantic, this value is 2 and for the Indo-Pacific this value is 1.

You can pick out values in either basin by using ```.where()```. For example, if I wanted to pick the zonal velocity fields only in the Atlantic ocean I would do the following:

In [ ]:
v_atlantic=ds.v.where(ds.basin_mask==2)

In [ ]:
ds.basin_mask.plot()


In [ ]:
###YOUR CODE HERE
v_atl = ds.v.sel(yu_ocean=slice(-35, 70)).mean('time')
atl_mask = (ds.basin_mask==2)
atl_mask = atl_mask * ds.vmask
v_atl = v_atl.where(atl_mask)
T_yz = (v_atl * ds.dxu).sum('xu_ocean')
Psi = (T_yz * ds.dzt).cumsum('st_ocean') / 1e6

fig, ax = plt.subplots(figsize=(10, 6), dpi=300, subplot_kw={'facecolor': '0.5'})
ax.set_title('Atlantic Meridional Overturning Circulation')
f1 = ax.contourf(Psi['yu_ocean'], Psi['st_ocean'], Psi, levels=np.arange(-30,31, 3), extend='both', cmap=cmo.balance)
cbar = plt.colorbar(f1, ax=ax, orientation='horizontal', pad=0.1, shrink=0.80, aspect=50, fraction=0.05, label='Streamfunction (Sv)')
ax.invert_yaxis()
ax.set_xlabel('Latitude')
ax.set_ylabel('Depth (m)')
fig.tight_layout()
fig.savefig('images/Atlantic_MOC.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
Psi_26 = Psi.sel(yu_ocean=26.5, method='nearest').max('st_ocean')
float(Psi_26)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), dpi=300, subplot_kw={'facecolor': '0.5'})
ax.set_title('Atlantic Meridional Overturning Circulation')
f1 = ax.pcolormesh(Psi['yu_ocean'], Psi['st_ocean'], Psi, cmap='turbo', vmin=-6, vmax=30)
cbar = plt.colorbar(f1, ax=ax, orientation='horizontal', pad=0.1, shrink=0.80, aspect=50, fraction=0.05, label='Streamfunction (Sv)')
ax.invert_yaxis()
ax.set_xlabel('Latitude')
ax.set_ylabel('Depth (m)')
fig.tight_layout()
fig.savefig('images/Atlantic_MOC_pcolormesh.png', dpi=300, bbox_inches='tight', pad_inches=0.1)

In [ ]:
v_ip = ds.v.sel(yu_ocean=slice(-30, 70)).mean('time')
ind_mask = (ds.basin_mask==1)
ind_mask = ind_mask.where(ds.vmask==1)
v_ip = v_ip.where(ind_mask)
T_yz = (v_ip * ds.dxu).sum('xu_ocean')
Psi = (T_yz * ds.dzt).cumsum('st_ocean') / 1e6

fig, ax = plt.subplots(figsize=(10, 6), dpi=300, subplot_kw={'facecolor': '0.5'})
ax.set_title('Indian-Pacific Meridional Overturning Circulation')
f1 = ax.contourf(Psi['yu_ocean'], Psi['st_ocean'], Psi, levels=np.arange(-30,31, 3), extend='both', cmap=cmo.balance)
cbar = plt.colorbar(f1, ax=ax, orientation='horizontal', pad=0.1, shrink=0.80, aspect=50, fraction=0.05, label='Streamfunction (Sv)')
ax.invert_yaxis()
ax.set_xlabel('Latitude')
ax.set_ylabel('Depth (m)')
fig.tight_layout()
fig.savefig('images/Indian-Pacific_MOC.png', dpi=300, bbox_inches='tight', pad_inches=0.1)


#### Take a moment to look at your plots
Write down some general notes (in this notebook or anywhere else that you prefer taking notes) describing what you see. For example, where are the general areas of maximum and minimum values for each of the plots. Can you point out both limbs of the overturning in your Atlantic plot? What do you think is happening in the Indo-Pacific?

---

## 4. Meridional Heat Transport (Bonus question)
In this part, you'll make a single plot of the time mean global meridional (advective) ocean heat transport (in Petawatts [1 PW = 1x10$^{15}$W]). Your plot should be a line plot of ocean heat transport vs. latitude (that means you need to integrate zonally and vertically).

The meridional heat transport is <br>

$$H(y) = \rho_{0}C_{p}\int_{-H}^{0} \int_{xe}^{xw} v\theta\, dxdz$$

Some constants that may be helpful:<br>
$\rho_{0}$=1027 kg m$^{-3}$<br>
$C_{p}$=3990 J kg$^{-1}$ K$^{-1}$<br>

><font color='Red'><b>Note: </b></font>Before you do your integral, you need to interpolate temperature onto the same points as the velocity field. The code to do that is below:

In [ ]:
theta=grid.interp(ds.temp, axis=('x','y'))

In [ ]:
###YOUR CODE HERE
rho0 = 1027.
Cp = 3990.
to_PW = 1e-15

v_mean = ds.v.mean('time')
theta_interp = (ds.temp.mean('time')).interp(xt_ocean=ds.xu_ocean, yt_ocean=ds.yu_ocean)
#.rename({'xt_ocean':'xu_ocean', 'yt_ocean':'yu_ocean'})

# Mask land
v_masked = v_mean * ds.vmask
theta_masked = theta_interp * ds.vmask

T_yz = (v_masked * theta_masked * ds.dxu).sum('xu_ocean')
H = (T_yz * ds.dzt).sum('st_ocean') * (rho0 * Cp * to_PW)


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,6), dpi=300)
ax.plot(H['yu_ocean'], H)
ax.set_title('Global Meridional Ocean Heat Transport')
ax.set_xlabel('Latitude')
ax.set_ylabel('Ocean Heat Transport (PW)')
ax.grid()
fig.tight_layout()
fig.savefig('images/Global_heat_transport.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
#fig.show()


In [ ]:
ds.vmask.plot()

#### Take a moment to look at your plots
Write down some general notes describing what you see. What is the general shape of the global ocean heat transport plot? Where is it positive and negative? What are the maximum and minimum values?

In [ ]:
x = np.array([[1],[1]])
y = np.array([[2],[0.8]])

x_norm = np.linalg.norm(x, 2)
y_norm = np.linalg.norm(y, 2)

print(np.dot(x.T, y) / (y_norm))

print(x_norm)
print(y_norm)


